In [1]:
import requests
import pandas as pd

url = 'https://www.buda.com/api/v2/markets'
response = requests.get(url)

df_mercado = pd.DataFrame(response.json()['markets'])
df_mercado['name'].unique()

array(['btc-clp', 'btc-cop', 'eth-clp', 'eth-btc', 'btc-pen', 'eth-pen',
       'eth-cop', 'bch-btc', 'bch-clp', 'bch-cop', 'bch-pen', 'btc-ars',
       'eth-ars', 'bch-ars', 'ltc-btc', 'ltc-clp', 'ltc-cop', 'ltc-pen',
       'ltc-ars', 'usdc-clp', 'usdc-cop', 'usdc-pen', 'usdc-ars',
       'btc-usdc', 'usdt-usdc'], dtype=object)

In [2]:
market_id = 'btc-clp'
url_market = f'https://www.buda.com/api/v2/markets/{market_id}/ticker'
def spread(market_id):
    response = requests.get(url_market)
    if response.status_code == 200:
        data = response.json()
        spread = float(data['ticker']['min_ask'][0]) - float(data['ticker']['max_bid'][0])
        return spread
    else:
        print(f"ERROR: No se puede obtener datos. ")
        return 0

df_spread = spread(market_id)
print(f"Spread para {market_id}: {df_spread}")


Spread para btc-clp: 218754.0


In [3]:
import requests

def spread_mercados():
    # Obtener la lista de mercados
    markets_url = 'https://www.buda.com/api/v2/markets'
    markets_response = requests.get(markets_url)
    
    if markets_response.status_code == 200:
        markets_data = markets_response.json()
        spreads = {}

        for mercado in markets_data.get('markets', []):
            market_id = mercado.get('id')
            if market_id:

                ticker_url = f'https://www.buda.com/api/v2/markets/{market_id}/ticker'
                ticker_response = requests.get(ticker_url)
                
                if ticker_response.status_code == 200:
                    ticker_data = ticker_response.json().get('ticker', {})
                    
                    if 'min_ask' in ticker_data and 'max_bid' in ticker_data:

                        min_ask = float(ticker_data['min_ask'][0])
                        max_bid = float(ticker_data['max_bid'][0])
                        spread = min_ask - max_bid
                        spreads[market_id] = spread
                else:
                    print("ERROR: No se puede obtener datos del ticker.")
                    
        return spreads
    else:
        print("ERROR: No se puede obtener datos de los mercados.")
        return None


spreads_mercados = spread_mercados()

if spreads_mercados:
    print("Lista de spreads:")
    for market_id, spread in spreads_mercados.items():
        print(f"{market_id}: {spread}")
else:
    print("ERROR: No se puede obtener la lista.")



Lista de spreads:
BTC-CLP: 218754.0
BTC-COP: 1462857.019999981
ETH-CLP: 34221.0
ETH-BTC: 0.00047536000000000106
BTC-PEN: 4265.649999999994
ETH-PEN: 353.8899999999994
ETH-COP: 6006.3300000000745
BCH-BTC: 7.085999999999985e-05
BCH-CLP: 4301.0
BCH-COP: 27469.640000000014
BCH-PEN: 48.50999999999999
BTC-ARS: 0.0
ETH-ARS: 0.0
BCH-ARS: 0.0
LTC-BTC: 4.4800000000000005e-05
LTC-CLP: 516.8600000000006
LTC-COP: 777.0
LTC-PEN: 9.310000000000002
LTC-ARS: 0.0
USDC-CLP: 3.9300000000000637
USDC-COP: 16.0
USDC-PEN: 0.06999999999999984
USDC-ARS: 0.0
BTC-USDC: 302.3199999999997
USDT-USDC: 0.0005999999999999339


In [4]:
import requests
import time

def obtener_spread(market_id):
    url = f'https://www.buda.com/api/v2/markets/{market_id}/ticker'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json().get('ticker', {})
        last_price = float(data.get('last_price', [0])[0])
        max_bid = float(data.get('max_bid', [0])[0])
        spread = last_price - max_bid
        return spread
    else:
        print(f"Error al obtener el spread para {market_id}. Código de estado: {response.status_code}")
        return None

def obtener_spread_alerta():
    try:
        with open('spread_alerta.txt', 'r') as file:
            spread_alerta = float(file.read())
        return spread_alerta
    except FileNotFoundError:
        return None
    except Exception as e:
        print(f"Error al leer el archivo de spread de alerta: {e}")
        return None

def guardar_spread_alerta(spread_alerta):
    try:
        with open('spread_alerta.txt', 'w') as file:
            file.write(str(spread_alerta))
        print(f"Spread de alerta guardado: {spread_alerta}")
    except Exception as e:
        print(f"Error al guardar el spread de alerta en el archivo: {e}")

def realizar_polling(market_id, intervalo_polling=60):
    while True:
        spread_actual = obtener_spread(market_id)
        spread_alerta = obtener_spread_alerta()

        if spread_actual is not None and spread_alerta is not None:
            if spread_actual > spread_alerta:
                print(f"¡Alerta! El spread actual ({spread_actual}) es mayor que el spread de alerta ({spread_alerta}).")
            elif spread_actual < spread_alerta:
                print(f"¡Alerta! El spread actual ({spread_actual}) es menor que el spread de alerta ({spread_alerta}).")

        time.sleep(intervalo_polling)

market_id_ejemplo = 'btc-clp'
spread_alerta_ejemplo = 10000  

guardar_spread_alerta(spread_alerta_ejemplo)
realizar_polling(market_id_ejemplo)


Spread de alerta guardado: 10000
¡Alerta! El spread actual (207663.0) es mayor que el spread de alerta (10000.0).
